<a href="https://colab.research.google.com/github/gtbnhyujmj/-Good-Auto_Shorts_Maker/blob/main/%5BV1%5D_Stickered_Movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1：引入套件 + 初始化設定

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import random
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.font_manager as fm

In [4]:
# 設定輸出影片大小
IMG_SIZE = (1080, 1920)
FONT_SIZE = 100

TOP_OFFSET = 150
RIGHT_MARGIN = 60

FPS = 30
DURATION = 3  # 秒數

# Step 2：隨機抽 PNG 圖片的工具

In [5]:
import os
import random
from PIL import Image
import cv2

def extract_middle_frame_as_pil(mp4_path):
    cap = cv2.VideoCapture(mp4_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {mp4_path}")

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    mid_frame_idx = frame_count // 2
    cap.set(cv2.CAP_PROP_POS_FRAMES, mid_frame_idx)

    ret, frame = cap.read()
    cap.release()

    if not ret:
        raise ValueError(f"Cannot read frame from video: {mp4_path}")

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return Image.fromarray(frame_rgb)

def get_random_images_from_folders(folder_paths, count=1):
    selected_images = []
    for folder in folder_paths:
        if not os.path.isdir(folder):
            print(f"Warning: {folder} is not a valid directory.")
            continue

        candidates = [f for f in os.listdir(folder) if f.lower().endswith(('.png', '.mp4'))]
        if not candidates:
            print(f"No valid .png or .mp4 files found in {folder}")
            continue

        sampled_files = random.sample(candidates, min(count, len(candidates)))
        for filename in sampled_files:
            file_path = os.path.join(folder, filename)
            if filename.lower().endswith('.png'):
                image = Image.open(file_path).convert("RGBA")
            elif filename.lower().endswith('.mp4'):
                image = extract_middle_frame_as_pil(file_path).convert("RGBA")
            else:
                continue
            selected_images.append(image)

    return selected_images

# Step 3：自動尋找中文字體（跨平台）

In [6]:
def find_chinese_font():
    font_candidates = fm.findSystemFonts(fontpaths=["/usr/share/fonts", "/System/Library/Fonts", "/content/drive/MyDrive/shorts/font"])
    for font_path in font_candidates:
        if "NotoSansCJK" in font_path or "Taipei" in font_path or "DFKai" in font_path:
            return font_path
    return None

# Step 4：產生文字貼紙圖（文字 → PNG）

In [7]:
def create_text_image(text, font_path, size=IMG_SIZE):
    img = Image.new("RGBA", size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(font_path, FONT_SIZE)

    text_size = draw.textsize(text, font=font)
    position = (size[0] - text_size[0] - RIGHT_MARGIN, TOP_OFFSET)
    draw.text(position, text, fill=(0, 255, 255, 255), font=font)
    return img

# Step 5：圖片疊圖處理（透明度合成）

In [8]:
def blend_with_alpha(base_img, overlays):
    base_rgba = cv2.cvtColor(base_img, cv2.COLOR_BGR2RGBA)
    for overlay in overlays:
        overlay_rgba = np.array(overlay.resize((base_img.shape[1], base_img.shape[0]))).astype(np.float32)
        alpha_overlay = overlay_rgba[..., 3:] / 255.0
        alpha_base = base_rgba[..., 3:] / 255.0
        out_alpha = alpha_overlay + alpha_base * (1 - alpha_overlay)
        base_rgba[..., :3] = (overlay_rgba[..., :3] * alpha_overlay + base_rgba[..., :3] * alpha_base * (1 - alpha_overlay)) / (out_alpha + 1e-6)
        base_rgba[..., 3:] = out_alpha * 255
    return cv2.cvtColor(base_rgba.astype(np.uint8), cv2.COLOR_RGBA2BGR)

# Step 6：製作影片（依照每個文字產生 1 支影片）

In [9]:
def create_short_video(text_img, overlay_imgs, output_path):
    width, height = IMG_SIZE
    total_frames = FPS * DURATION
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, FPS, (width, height))

    for _ in range(total_frames):
        blank = np.zeros((height, width, 3), dtype=np.uint8)
        frame = blend_with_alpha(blank, overlay_imgs + [text_img])
        out.write(frame)

    out.release()
    print(f"已輸出：{output_path}")

# Step 7：主控流程（串起所有步驟）

In [10]:
# 路徑設定
BACKGROUND_DIRS = ["/content/drive/MyDrive/shorts/backgrounds_MP4", "/content/drive/MyDrive/shorts/characters", "/content/drive/MyDrive/shorts/PNG_Overlayers"]
TEXT_STICKER_DIR = "/content/drive/MyDrive/shorts/Text_Overlayers"
OUTPUT_DIR = "/content/drive/MyDrive/shorts/movie01"
os.makedirs(OUTPUT_DIR, exist_ok=True)

font_path = find_chinese_font()
text_files = [f for f in os.listdir(TEXT_STICKER_DIR) if f.endswith('.png')]

for text_file in text_files:
    text_path = os.path.join(TEXT_STICKER_DIR, text_file)
    text_overlay = Image.open(text_path).convert("RGBA")

    random_layers = get_random_images_from_folders(BACKGROUND_DIRS)
    output_path = os.path.join(OUTPUT_DIR, f"{text_file.replace('.png', '')}.mp4")
    create_short_video(text_overlay, random_layers, output_path)

已輸出：/content/drive/MyDrive/shorts/movie01/text_1.mp4
已輸出：/content/drive/MyDrive/shorts/movie01/text_2.mp4
已輸出：/content/drive/MyDrive/shorts/movie01/text_0.mp4
